# BIG DATA : DATA PROCESSING AND ANALYSIS OF BRISTOL COVID-19 DATA



In [1]:
# entry point to using spark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName('COVID-19 Data') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/18 17:21:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
#checking which version of pyspark is running
spark.version

'3.2.1'

In [5]:
# read data as a pyspark dataframe
df = (spark.read
          .format("csv")
          .options(delimiter=";", header=True)
          .load("covid-19-vaccinations.csv"))

In [6]:
# convert dataframe to pandas dataframe
df.toPandas()

,Area name,Vaccination date,Age group,Cumulative first dose vaccinations,Cumulative second dose vaccinations,geo_point_2d
0,South Gloucestershire,2021-07-07,55_59,19314,18704,"51.5468677188,-2.47061248413"
1,South Gloucestershire,2021-06-30,45_49,16964,11411,"51.5468677188,-2.47061248413"
2,South Gloucestershire,2021-06-30,50_54,19533,18814,"51.5468677188,-2.47061248413"
3,South Gloucestershire,2021-06-30,55_59,19298,18617,"51.5468677188,-2.47061248413"
4,South Gloucestershire,2021-07-06,75_79,11167,11080,"51.5468677188,-2.47061248413"
...,...,...,...,...,...,...
635524,"Bristol, City of",2022-02-13,18_24,49507,43546,"51.454513,-2.58791"
635525,"Bristol, City of",2022-02-13,60_64,20099,19766,"51.454513,-2.58791"
635526,North Somerset,2022-02-13,18_24,12457,11577,"51.3892298341,-2.80066579483"
635527,England,2022-02-13,16_17,924735,669519,None


In [8]:
# show statistics
df.describe().show()

+-------+--------------------+----------------+---------+----------------------------------+-----------------------------------+--------------------+
|summary|           Area name|Vaccination date|Age group|Cumulative first dose vaccinations|Cumulative second dose vaccinations|        geo_point_2d|
+-------+--------------------+----------------+---------+----------------------------------+-----------------------------------+--------------------+
|  count|              635529|          635529|   635529|                            635529|                             635529|              429806|
|   mean|                null|            null|     null|                 503306.0769988466|                 250231.77263665388|                null|
| stddev|                null|            null|     null|                  977247.718181228|                  660790.6894958996|                null|
|    min|Bath and North Ea...|      2020-12-08|    12_15|                                 0|        

In [9]:
#show statistics as a pandas dataframe
df.describe().toPandas()

,summary,Area name,Vaccination date,Age group,Cumulative first dose vaccinations,Cumulative second dose vaccinations,geo_point_2d
0,count,635529,635529,635529,635529,635529,429806
1,mean,None,None,None,503306.0769988466,250231.77263665388,None
2,stddev,None,None,None,977247.718181228,660790.6894958996,None
3,min,Bath and North East Somerset,2020-12-08,12_15,0,0,"51.3559707863,-2.47520121048"
4,max,South Gloucestershire,2022-03-02,90+,999963,999949,"51.5468677188,-2.47061248413"


In [11]:
# check the datatypes of the dataframe columns
df.dtypes

[('Area name', 'string'),
 ('Vaccination date', 'string'),
 ('Age group', 'string'),
 ('Cumulative first dose vaccinations', 'string'),
 ('Cumulative second dose vaccinations', 'string'),
 ('geo_point_2d', 'string')]

In [12]:
# to convert columns to their appropriate data format
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,DateType,IntegerType,FloatType,DecimalType

df2 = df.withColumn("Area name",col("Area name").cast(StringType())) \
    .withColumn("Vaccination date",col("Vaccination date").cast(DateType())) \
    .withColumn("Age group",col("Age group").cast(StringType())) \
    .withColumn("Cumulative first dose vaccinations",col("Cumulative first dose vaccinations").cast(IntegerType())) \
    .withColumn("Cumulative second dose vaccinations",col("Cumulative second dose vaccinations").cast(IntegerType())) \
    .withColumn("geo_point_2d",col("geo_point_2d").cast(StringType()))
df2.dtypes

[('Area name', 'string'),
 ('Vaccination date', 'date'),
 ('Age group', 'string'),
 ('Cumulative first dose vaccinations', 'int'),
 ('Cumulative second dose vaccinations', 'int'),
 ('geo_point_2d', 'string')]

In [13]:
#describing Cumulative first dose vaccinations
df.describe(['Cumulative first dose vaccinations']).show()

+-------+----------------------------------+
|summary|Cumulative first dose vaccinations|
+-------+----------------------------------+
|  count|                            635529|
|   mean|                 503306.0769988466|
| stddev|                  977247.718181228|
|    min|                                 0|
|    max|                            999963|
+-------+----------------------------------+



In [14]:
#describing Cumulative second dose vaccinations
df.describe(['Cumulative second dose vaccinations']).show()

+-------+-----------------------------------+
|summary|Cumulative second dose vaccinations|
+-------+-----------------------------------+
|  count|                             635529|
|   mean|                 250231.77263665388|
| stddev|                  660790.6894958996|
|    min|                                  0|
|    max|                             999949|
+-------+-----------------------------------+



In [17]:
# correlation before dropping null values
df = df2.toPandas()
co=df['Cumulative first dose vaccinations'].corr(df['Cumulative second dose vaccinations'])
print(co)


0.804078154408801


0.804078154408801


In [16]:
# check where null values exist
from pyspark.sql.functions import isnull, when, count
df2.select([count(when(isnull(c), c)).alias(c) for c in df2.columns]).show()

+---------+----------------+---------+----------------------------------+-----------------------------------+------------+
|Area name|Vaccination date|Age group|Cumulative first dose vaccinations|Cumulative second dose vaccinations|geo_point_2d|
+---------+----------------+---------+----------------------------------+-----------------------------------+------------+
|        0|               0|        0|                                 0|                                  0|      205723|
+---------+----------------+---------+----------------------------------+-----------------------------------+------------+



In [18]:
# remove null values
df2 = df2.replace('?', None) \
    .dropna(how='any')

# Drop unnecessary columns
# df2 = df2.drop('')
# df2 = df2.drop('')

df2.toPandas()

,Area name,Vaccination date,Age group,Cumulative first dose vaccinations,Cumulative second dose vaccinations,geo_point_2d
0,South Gloucestershire,2021-07-07,55_59,19314,18704,"51.5468677188,-2.47061248413"
1,South Gloucestershire,2021-06-30,45_49,16964,11411,"51.5468677188,-2.47061248413"
2,South Gloucestershire,2021-06-30,50_54,19533,18814,"51.5468677188,-2.47061248413"
3,South Gloucestershire,2021-06-30,55_59,19298,18617,"51.5468677188,-2.47061248413"
4,South Gloucestershire,2021-07-06,75_79,11167,11080,"51.5468677188,-2.47061248413"
...,...,...,...,...,...,...
429801,North Somerset,2022-02-15,55_59,15416,15242,"51.3892298341,-2.80066579483"
429802,North Somerset,2022-02-15,70_74,13457,13393,"51.3892298341,-2.80066579483"
429803,"Bristol, City of",2022-02-13,18_24,49507,43546,"51.454513,-2.58791"
429804,"Bristol, City of",2022-02-13,60_64,20099,19766,"51.454513,-2.58791"


In [19]:
# show statistics
df2.describe().show()

+-------+--------------------+---------+----------------------------------+-----------------------------------+--------------------+
|summary|           Area name|Age group|Cumulative first dose vaccinations|Cumulative second dose vaccinations|        geo_point_2d|
+-------+--------------------+---------+----------------------------------+-----------------------------------+--------------------+
|  count|              429806|   429806|                            429806|                             429806|              429806|
|   mean|                null|     null|                 9846.110542896098|                 5085.0945286943415|                null|
| stddev|                null|     null|                 7064.787413481537|                  5566.919561870152|                null|
|    min|Bath and North Ea...|    12_15|                                 0|                                  0|51.3559707863,-2....|
|    max|South Gloucesters...|      90+|                             

In [20]:
#show statistics as a pandas dataframe
df2.describe().toPandas()

,summary,Area name,Age group,Cumulative first dose vaccinations,Cumulative second dose vaccinations,geo_point_2d
0,count,429806,429806,429806,429806,429806
1,mean,None,None,9846.110542896098,5085.0945286943415,None
2,stddev,None,None,7064.787413481537,5566.919561870152,None
3,min,Bath and North East Somerset,12_15,0,0,"51.3559707863,-2.47520121048"
4,max,South Gloucestershire,90+,49936,44050,"51.5468677188,-2.47061248413"


In [21]:
#describing Cumulative first dose vaccinations
df2.describe(['Cumulative first dose vaccinations']).show()

+-------+----------------------------------+
|summary|Cumulative first dose vaccinations|
+-------+----------------------------------+
|  count|                            429806|
|   mean|                 9846.110542896098|
| stddev|                 7064.787413481537|
|    min|                                 0|
|    max|                             49936|
+-------+----------------------------------+



In [22]:
#describing Cumulative second dose vaccinations
df2.describe(['Cumulative second dose vaccinations']).show()

+-------+-----------------------------------+
|summary|Cumulative second dose vaccinations|
+-------+-----------------------------------+
|  count|                             429806|
|   mean|                 5085.0945286943415|
| stddev|                  5566.919561870152|
|    min|                                  0|
|    max|                              44050|
+-------+-----------------------------------+



In [23]:
# correlation after dropping null values
df = df2.toPandas()
co=df['Cumulative first dose vaccinations'].corr(df['Cumulative second dose vaccinations'])
print(co)


0.6847338260438598


In [24]:
#  creates a temporary view covid_table for SQL query
df2.createOrReplaceTempView("covid_table")

In [25]:
# query to check how many rows exists
spark.sql("SELECT count(*) as count from covid_table").show()

+------+
| count|
+------+
|429806|
+------+



In [34]:
# selecting age group, cumulative sum of first and second dose
spark.sql("SELECT  `Age group`, SUM(`Cumulative first dose vaccinations`) ,SUM(`Cumulative second dose vaccinations`) from covid_table group by `Age group` order by  SUM(`Cumulative first dose vaccinations`) DESC ,SUM(`Cumulative second dose vaccinations`) DESC").show()

+---------+---------------------------------------+----------------------------------------+
|Age group|sum(Cumulative first dose vaccinations)|sum(Cumulative second dose vaccinations)|
+---------+---------------------------------------+----------------------------------------+
|    55_59|                              428601746|                               227974065|
|    50_54|                              410709206|                               218566884|
|    60_64|                              333325037|                               183557004|
|    45_49|                              327176498|                               158622129|
|    40_44|                              316506581|                               141672943|
|    70_74|                              314305569|                               196906671|
|    35_39|                              295975057|                               122080783|
|    18_24|                              293994499|                   

# Information
People that belong to age group 50-64, had the highest number of vaccinations
People aged below 17 had the lowest vaccinations and No vaccination dose were recorded for people less than 12

# Knowledge
People in ages 50-64 had more recorded vaccinations because they are more susceptible to covid outbreak
People younger than 18 were vaccinated less because the older ones are more prone to getting the covid because of weaker immune system therefore there is a need to raise awareness for more younger people to take their jab



In [38]:
# select table of cumulative first and second dose and group by year
spark.sql("SELECT  Year(`Vaccination date`), SUM(`Cumulative first dose vaccinations`) ,SUM(`Cumulative second dose vaccinations`) from covid_table group by  Year(`Vaccination date`) order by  SUM(`Cumulative first dose vaccinations`) DESC ,SUM(`Cumulative second dose vaccinations`) DESC").show()

+----------------------+---------------------------------------+----------------------------------------+
|year(Vaccination date)|sum(Cumulative first dose vaccinations)|sum(Cumulative second dose vaccinations)|
+----------------------+---------------------------------------+----------------------------------------+
|                  2021|                             4173346133|                              2132996081|
|                  2022|                               56211346|                                52607947|
|                  2020|                                2359909|                                     111|
+----------------------+---------------------------------------+----------------------------------------+



# Information
2021 had the highest number of vaccinations
111 people took their second dose in 2020
# Knowledge
More people were increasingly aware of covid scare
Fewer people took their first and second jab because vaccinations were introduced late 2020





In [37]:
# select table of cumulative first and second dose and group by year
spark.sql("SELECT  `Area name`, SUM(`Cumulative first dose vaccinations`) ,SUM(`Cumulative second dose vaccinations`) from covid_table group by `Area name` order by  SUM(`Cumulative first dose vaccinations`) DESC ,SUM(`Cumulative second dose vaccinations`) DESC ").show()

+--------------------+---------------------------------------+----------------------------------------+
|           Area name|sum(Cumulative first dose vaccinations)|sum(Cumulative second dose vaccinations)|
+--------------------+---------------------------------------+----------------------------------------+
|    Bristol, City of|                             1737607833|                               861039060|
|South Gloucesters...|                             1096397334|                               573693170|
|      North Somerset|                              812452204|                               441900472|
|Bath and North Ea...|                              585460017|                               308971437|
+--------------------+---------------------------------------+----------------------------------------+



# Information
Bristol City had the highest number of recorded first and second dose
# Knowledge
More people live in bristol city than other areas , hence their high recorded vaccinations done




In [43]:
# no of times people in an age-group were registered to take their vaccine
# despite registering more , they averaged less vaccination shots than others
spark.sql("SELECT  `Age group`, COUNT(`Age group`), AVG(`Cumulative first dose vaccinations`) ,AVG(`Cumulative second dose vaccinations`) from covid_table group by `Age group` order by COUNT(`Age group`) DESC ").show()

+---------+----------------+---------------------------------------+----------------------------------------+
|Age group|count(Age group)|avg(Cumulative first dose vaccinations)|avg(Cumulative second dose vaccinations)|
+---------+----------------+---------------------------------------+----------------------------------------+
|      90+|           34200|                     2357.9977192982456|                      1567.8944736842104|
|    18_24|           33807|                      8696.261099772237|                      3105.5248912947022|
|    85_89|           33048|                      4160.381475429678|                      2796.1486625514403|
|    80_84|           30253|                     6829.6352097312665|                       4605.021816018246|
|    25_29|           30241|                      7989.970272147085|                       3168.679375682021|
|    30_34|           29958|                      9733.568328993924|                       3868.541057480473|
|    35_39

# Information
People aged 90+ were registered more times to get their vaccines
People aged 90+ had one of the least averaged first and second dose vaccinations

# Knowledge
People aged 90+ were encouraged to come register the most due to being prone to the virus
Though they were registered the most, the had one of the least vaccinations per day due to the population of people in that age group





In [40]:
# cumulative sum of first dose vaccinations vs second dose vaccinations
spark.sql("SELECT  SUM(`Cumulative first dose vaccinations`) ,SUM(`Cumulative second dose vaccinations`) from covid_table ").show()

+---------------------------------------+----------------------------------------+
|sum(Cumulative first dose vaccinations)|sum(Cumulative second dose vaccinations)|
+---------------------------------------+----------------------------------------+
|                             4231917388|                              2185604139|
+---------------------------------------+----------------------------------------+



# Information
First dose vaccinations were x2 more than second vaccinations

# Knowledge
During the heat and initial scare of the pandemic, people took their first dose vaccinations and a fraction decided not to take the second after covid cases reduced.





In [42]:
spark.sql("SELECT MONTH(`vaccination date`), YEAR(`vaccination date`), count(`vaccination date`)  from covid_table group by MONTH(`vaccination date`),YEAR(`vaccination date`) order by count(`vaccination date`) DESC ").show()

+-----------------------+----------------------+-----------------------+
|month(vaccination date)|year(vaccination date)|count(vaccination date)|
+-----------------------+----------------------+-----------------------+
|                      5|                  2021|                  74566|
|                      6|                  2021|                  74423|
|                      4|                  2021|                  68484|
|                      3|                  2021|                  59089|
|                      7|                  2021|                  54665|
|                      2|                  2021|                  38919|
|                      1|                  2021|                  25718|
|                      8|                  2021|                  16497|
|                     12|                  2020|                   5105|
|                     10|                  2021|                   2108|
|                     12|                  2021|   

# Information
May and June 2021 had the highest recorded covid vaccinations

#Knowledge
May and June 2021 possibly had the highest record covid vaccinations due to more vaccines available and people willing to take.



